# Code to create the Blog using Audrey Feldroy's Basic Blog Code

Blog site code builder for the Computer Blog

This code is taken from https://github.com/audreyfeldroy/audrey.feldroy.com

I have adapted this code so that it pulls from local css files and Javascript. It will potentially not have access to an Internet connected server so I need the code to still work. I have also pulled across the prototype from Audrey, that starts to use Tailwind CSS to make the Index page display dense text.
I have also changed some of the names of the Notebooks and I am now writing my own Today I Learnt Notebooks etc.

In [1]:
#| default_exp main

In [2]:
#| export
from datetime import datetime
from execnb.nbio import read_nb
from nb2fasthtml.core import render_code_output
from fastcore.utils import *
from fasthtml.common import *
from fasthtml.jupyter import *
from importlib.metadata import distributions
from IPython.display import display, HTML
from monsterui import franken
from monsterui.all import Theme
from mistletoe import markdown
from mistletoe.html_renderer import block_token, HtmlRenderer
import pygments
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter

In [3]:
#| export
def get_nb_paths(): 
    root = Path() if IN_NOTEBOOK else Path("nbs/")
    return L(root.glob("*.ipynb")).sorted(reverse=True)

In [4]:
#| export
nbpaths = get_nb_paths()
nbpaths

(#8) [Path('UndatedTestNotebook.ipynb'),Path('2025-03-26-LMStudioAndFabric.ipynb'),Path('2025-03-25-PythonScripts.ipynb'),Path('2025-03-24-WSL2Commands.ipynb'),Path('2025-03-23-UbuntuServerInstall.ipynb'),Path('2025-03-21-UsingPodmanInUbuntu.ipynb'),Path('2025-03-20-Useful-Linux-Commands.ipynb'),Path('2025-03-19-ComputerProjectBlogCode.ipynb')]

In [5]:
IN_NOTEBOOK

True

## Setup

In [6]:
#| export
# app,rt = fast_app(pico=False)
app,rt = fast_app(
    pico=False,
    ws_hdr=True,
    live=False,
    default_hdrs=False,
    hdrs=(
#        Link(rel='stylesheet', href='static/pico.min.css'),
        Link(rel="stylesheet", href="static/tailwind.css"),
        Script(src="static/htmx.min.js"),
        Script(src="static/fasthtml.js"),
        Script(src="static/surreal.js"),
        Script(src="static/script.js"),
        Script(src="static/index.global.js"),
        Meta(charset='utf-8'),
#        Script(type='module')
    ),
)

In [7]:
server = JupyUvi(app)

In [8]:
# server.stop()

## Utilities

In [9]:
#| export
def get_nb_paths(): 
    root = Path() if IN_NOTEBOOK else Path("nbs/")
    return L(root.glob("*.ipynb")).sorted(reverse=True)

In [10]:
nb_paths = get_nb_paths()
nb_paths

(#8) [Path('UndatedTestNotebook.ipynb'),Path('2025-03-26-LMStudioAndFabric.ipynb'),Path('2025-03-25-PythonScripts.ipynb'),Path('2025-03-24-WSL2Commands.ipynb'),Path('2025-03-23-UbuntuServerInstall.ipynb'),Path('2025-03-21-UsingPodmanInUbuntu.ipynb'),Path('2025-03-20-Useful-Linux-Commands.ipynb'),Path('2025-03-19-ComputerProjectBlogCode.ipynb')]

In [11]:
#| export
def get_title_and_desc(fpath):
    nb = read_nb(fpath)
    title = nb.cells[0].source.lstrip("# ")
    desc = nb.cells[1].source
    return title,desc

In [12]:
get_title_and_desc(nb_paths[0])

('Test Notebook with no date prefix',
 'This is a test to see whether I can have an undated notebook.')

In [13]:
#| export
def NBLink(title, desc, href, date):
    return Div(
        A(f"{date:%b %-d} • {title}", href=href, cls="text-md font-semibold hover:text-blue-600 no-underline block"),
        P(desc, cls="text-s text-gray-500 mt-0.5 mb-2"),
        cls="py-1 break-inside-avoid")

In [14]:
#| export
def mk_nblink_from_nbpath(nbpath):
    date = get_date_from_iso8601_prefix(nbpath.name) or datetime.now()
    return NBLink(*get_title_and_desc(nbpath), href=f'/nbs/{nbpath.name[:-6]}', date=date)

In [15]:
#| export
def get_date_from_iso8601_prefix(fname):
    "Gets date from first 10 chars YYYY-MM-DD of `fname`, where `fname` is like `2025-01-12-Get-Date-From-This.whatever"
    try:
        return datetime.fromisoformat(str(fname)[0:10])
    except ValueError: return datetime.now()

In [16]:
date = get_date_from_iso8601_prefix(nb_paths[0].name)
date

datetime.datetime(2025, 3, 24, 11, 43, 27, 916619)

In [17]:
date = get_date_from_iso8601_prefix(None)
date

datetime.datetime(2025, 3, 24, 11, 43, 28, 260715)

## Notebook Cards

In [18]:
#| export
def NBCard(title,desc,href,date):
    return A(
        franken.Card(
        franken.CardTitle(franken.H3(title)), 
        franken.P(f"{date:%a, %b %-d, %Y}", cls=franken.TextPresets.muted_sm),
        franken.P(desc),
        body_cls='space-y-2'
    ), href=href)

In [19]:
#| export
def mk_nbcard_from_nb_path(nb_path):
    date = get_date_from_iso8601_prefix(nb_path.name) or datetime.now()
    return NBCard(*get_title_and_desc(nb_path), href=f'/nbs/{nb_path.name[:-6]}', date=date)

## Nav

In [20]:
#| export
def InLi(linktuple):
    txt, href = linktuple
    return Li(A(txt, href=href), style="display:inline;margin-right:1em")

In [21]:
#| export
def InlineNav():
    nls = L(
        ("Alex Henderson Computer Project Blog", "http://ubuntu2410blog:8000"),
        ("GitHub repo for this site", "https://github.com/abhenderson")
    )
    return Nav(
        Ul(
            *nls.map(InLi),
            style="list-style:none;padding-left:0"
        ),
        aria_label="Main navigation",
        role="navigation"
    )

## Index Page

In [22]:
#| export
@rt
def index():
    nb_paths = get_nb_paths()
    return (
#        Theme.blue.headers(),
#        Script(src="https://unpkg.com/@tailwindcss/browser@4"),
#        Script(src="static/index.global.js"),
#        Script(src="/static/index.global.js"),
        Script(src="/static/tailwind.css"),
        Link(rel="stylesheet", href="static/tailwind.css"),
        Title("Alex Henderson Computer Project Blog"),
        Div(
            H1('Alex Henderson Computer Project Blog', cls="text-2x1 font-bold mb-2 dark:text-gray-100"),
            P(
                "Project Notebooks and Notes for my Computer Tasks ",
                A("github.com/abhenderson",
                  href="https://github.com/abhenderson",
                  cls="text-blue-600 hover:text-blue-800 dark:text-blue-400 dark:hover:text-blue-300"),
                cls="mb-4 text-gray-600 dark:text-gray-300 text-sm"
            ),
            Div(
                *nbpaths.map(mk_nblink_from_nbpath),
                cls="grid grid-cols-1 sm:grid-cols-2 lg:grid-cols-3 xl:grid-cols-4 gap-x-6 gap-y-2"
            ),
            cls="mx-auto px-2 py-4 dark:bg-gray-900 min-h-screen"
#            cls="md:flex"
#            cls="px-4 py-8 w-full columns-1 md:columns-2 lg:columns-3 gap-6"
        ),
    )

"""
        franken.Container(
#             InlineNav(),  # TODO: Fix incompatibilities with MonsterUI
            Div(
                franken.H1('Alex Henderson Computer Project Blog'),
                franken.P("Lessons I am learning from my various computer projects ", franken.A("About Me", href="/about")),
                franken.P("Notebooks I use to track my learning ", franken.A("Alex Project Blog", href="https://github.com/abhenderson"), cls="mb-6"),
            ),
            franken.Grid(*nb_paths.map(mk_nbcard_from_nb_path), cols_sm=1, cols_md=1, cols_lg=2, cols_xl=3)
        )
"""

def NBLink(title, desc, href, date):
    return Div(
        A(
            Div(
                f"{date:%b %-d}", 
                cls="text-xs font-medium text-gray-500 dark:text-gray-400"
            ),
            Div(
                title,
                cls="text-base font-medium leading-snug hover:text-blue-600 dark:text-gray-100 dark:hover:text-blue-400"
            ),
            P(
                desc,
                cls="text-xs text-gray-600 dark:text-gray-400 mt-0.5 line-clamp-2"
            ),
            href=href,
            cls="block no-underline hover:bg-gray-50 dark:hover:bg-gray-800 p-2 rounded transition-colors"
        ),
        cls="break-inside-avoid"
    )

## About
This is a test update to the file.

In [23]:
#| export
@rt
def about():
    nb_paths = get_nb_paths()
    return (
        Style(':root {font-family: system-ui, -apple-system, "Segoe UI", Roboto, "Helvetica Neue", sans-serif; color-scheme: light dark;} body {background-color: light-dark(#ffffff, #1a1a1a); color: light-dark(#1a1a1a, #ffffff);} p {line-height: 1.5;}'),
        Theme.blue.headers(),
#        InlineNav(),
        Title("About page"),
        franken.Container(
            InlineNav(),
            Div(
                franken.H1('About Page'), franken.P("My about page"), cls="mb-6"),
        ),
    )

## Notebook Cells

In [24]:
#| export
def StyledCode(c, style='monokai'):
    fm = HtmlFormatter(style=style, cssclass=style, prestyles="padding:10px 0;")
    h = highlight(c, PythonLexer(), fm)
    sd = fm.get_style_defs(f".{style}")
    return Style(sd), NotStr(h)

In [25]:
#| export
class MonsterHtmlRenderer(HtmlRenderer):
    def render_heading(self, token: block_token.Heading) -> str:
        template = '<h{level} class="uk-h{level}">{inner}</h{level}>'
        inner = self.render_inner(token)
        return template.format(level=token.level, inner=inner)

In [26]:
#| export
def StyledMd(m):
    return Safe(markdown(m, MonsterHtmlRenderer))

In [27]:
#| export
def StyledCell(c):
    if c.cell_type == "markdown": return StyledMd(c.source)
    if c.cell_type == "code": 
        if not c.outputs: return StyledCode(c.source)
        return StyledCode(c.source), render_code_output(c)

## Detail Page

In [28]:
#| export
@rt("/nbs/{name}")
def notebook(name:str):
    fname = f"{name}.ipynb" if IN_NOTEBOOK else f"nbs/{name}.ipynb"
    fpath = Path(fname)
    nb = read_nb(fpath)
    title = nb.cells[0].source.lstrip("# ")
    date = get_date_from_iso8601_prefix(fname.lstrip("nbs/"))
    desc = nb.cells[1].source
    if "MonsterUI" in title:
        return (
            Theme.slate.headers(),
#             InlineNav(),  # TODO: Fix incompatibilities with MonsterUI
            Title(title),
            franken.Container(
                Header(
                    # TODO: refactor Tailwind margin classes to use MonsterUI DivVStacked or DivFullySpaced
                    franken.H1(title, cls=("my-6",)),
                    franken.P(f"by Alex Henderson Blog | {date:%a, %b %-d, %Y}", cls=(franken.TextT.sm, franken.PaddingT.lg, "mb-6")),
                    franken.P(desc, cls=("mb-6",)),
                    Hr()
                ),
                *L(nb.cells[2:]).map(StyledCell),
                cls="space-y-5"
            )
    )
    return (
        Style(':root {font-family: system-ui, -apple-system, "Segoe UI", Roboto, "Helvetica Neue", sans-serif; color-scheme: light dark;} body {background-color: light-dark(#ffffff, #1a1a1a); color: light-dark(#1a1a1a, #ffffff);} p {line-height: 1.5;}'),
        InlineNav(),
        Title(title),
        Div(
            H1(title), # Title
            P(Small(f"by Alex Henderson Blog | {date:%a, %b %-d, %Y}")),
            P(desc),
            Hr(),
            *L(nb.cells[2:]).map(StyledCell),
            cls="space-y-5"
        )
    )

## Python Package Versions

In [29]:
#| export
@rt
def versions():
    dists = L([NS(name=dist.metadata['Name'], version=dist.version) for dist in distributions()]).sorted('name')
    dists = [Li(f'{d.name}: {d.version}') for d in dists]
    return (Title('Python Package Versions'),
        Style(':root {font-family: system-ui, -apple-system, "Segoe UI", Roboto, "Helvetica Neue", sans-serif; color-scheme: light dark;} body {background-color: light-dark(#ffffff, #1a1a1a); color: light-dark(#1a1a1a, #ffffff);} p {line-height: 1.5;}'),   
        Div(
            H1('Python Package Versions'),
            Ul(*dists)          
        )       
    )

## .well-known

In [30]:
#| export
@rt('/.well-known/{fname}')
def wellknown(fname: str):
    fpath = f"../.well-known/{fname}" if IN_NOTEBOOK else f".well-known/{fname}"
    return Path(fpath).read_text()

## Serve

In [31]:
#| export
serve()

## Export

To export this notebook as [audrey.feldroy.com's main.py](https://github.com/audreyfeldroy/audrey.feldroy.com/blob/main/main.py):

In [32]:
from nbdev.export import nb_export
# nb_export("2025-02-07-This-Site-Is-Now-Powered-by-This-Notebook-Part-6.ipynb", lib_path="..")
nb_export("2025-03-19-ComputerProjectBlogCode.ipynb", lib_path="..")

## Stop server 

In [ ]:
server.stop()